In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <BBC4A126-D15A-3802-AD26-108872BA781A> /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/O

In [21]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
if 'Survived' not in df_test.columns:
    df_test['Survived'] = 0


In [22]:
def Preprocess(df_train,df_test):
    df = pd.concat([df_train,df_test], axis = 0)
    df = df.drop(['Name', 'Ticket'], axis=1)
    df['Age'] = df['Age'].fillna(df.Age.mean())
    df['Cabin'] = df['Cabin'].fillna('X000')
    df['Embarked'] = df['Embarked'].fillna('X')
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    df['cabin_letter'] = df['Cabin'].str.extract(r'([a-zA-z]+)', expand=False)
    df['cabin_number'] = df['Cabin'].str.extract(r'(\d+)', expand=False)
    df = df.drop('Cabin', axis=1)
    df['cabin_number'] = df['cabin_number'].fillna('000')
    df = pd.get_dummies(df, columns=['cabin_letter'], prefix = 'cabin')
    df = pd.get_dummies(df, columns= ['Embarked'], prefix = 'Embarked')
    df = pd.get_dummies(df, columns= ['Sex'], prefix = 'Sex')
    df = df.drop('cabin_X', axis = 1)
    df = df.drop('Embarked_X', axis = 1)
    df_train = df[:len(df_train)]
    df_test = df[len(df_train):]

    df_test = df_test.drop('Survived', axis = 1)
    return df_train, df_test

In [23]:
train_df , test_df = Preprocess(df_train, df_test)

In [24]:
x = train_df.drop('Survived', axis=1)
y = train_df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2)
y_train = np.reshape(y_train, (-1, 1))

In [25]:
x_train.shape, y_train.shape

((712, 20), (712, 1))

In [26]:
model_1 = LogisticRegression()
model_1.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [28]:
y_pred = model_1.predict(x_test)

In [29]:
accuracy_score(y_test, y_pred)

0.7932960893854749